In [2]:
from flask import Flask, request, jsonify
import PyPDF2
import requests
import json
import logging
import time

app = Flask(__name__)

# --- Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- OpenRouter API Configuration ---
OPENROUTER_API_KEY = "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"
OPENROUTER_URL = "https://openrouter.ai/api/v1/chat/completions"
OPENROUTER_MODEL = "openai/gpt-4o"
HEADERS = {
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    "Content-Type": "application/json",
    "HTTP-Referer": "https://your-site.com",   # Optional
    "X-Title": "HiringHubAPI"                  # Optional
}

# --- Prompts ---
ANALYZE_PROMPT = """You are a professional career coach. Analyze the following resume text and provide specific, actionable suggestions to improve the candidate's skills and profile for the given job description:

Resume:
{}

Job Description:
{}"""

SKILL_IMPROVE_PROMPT = """You are a professional career coach. Review the following resume text and suggest targeted skill development opportunities to align with the provided job description:

Resume:
{}

Job Description:
{}"""

PERCENTAGE_MATCH_PROMPT = """You are a professional career coach. Evaluate the following resume text against the provided job description and provide ONLY a percentage match as a number (e.g., "75%" or "70-75%"). Do not include any explanations, analysis, or additional text - just the percentage.

Resume:
{}

Job Description:
{}"""

RECOMMEND_JOB_PROMPT = """Given the following CV, identify and output only the most relevant job title based on the skills, experience, and qualifications listed. Do not include any additional text, explanations, or special symbols—just the job title itself.

CV:
{}"""

# --- PDF Text Extraction ---
def extract_text_from_pdf(pdf_file):
    try:
        reader = PyPDF2.PdfReader(pdf_file)
        text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
        if not text.strip():
            return None, "No text found in PDF. Make sure it's a searchable document."
        return text.strip(), None
    except Exception as e:
        logger.error(f"PDF extraction error: {e}")
        return None, f"Error extracting text: {str(e)}"

# --- OpenRouter Request ---
def query_openrouter(prompt, retries=3, delay=5):
    payload = {
        "model": OPENROUTER_MODEL,
        "messages": [{"role": "user", "content": prompt}]
    }

    for attempt in range(retries):
        try:
            response = requests.post(OPENROUTER_URL, headers=HEADERS, data=json.dumps(payload))
            response.raise_for_status()
            data = response.json()
            if "choices" in data and data["choices"]:
                return data["choices"][0]["message"]["content"].strip(), None
            return None, "No response content from OpenRouter."
        except Exception as e:
            logger.error(f"[Attempt {attempt + 1}] API call failed: {e}")
            time.sleep(delay)
    return None, "Failed to reach OpenRouter after multiple attempts."

# --- API Routes ---

@app.route("/recommend_job", methods=["POST"])
def recommend_job():
    file = request.files.get("resume")
    if not file or not file.filename.endswith(".pdf"):
        return jsonify({"error": "Please upload a valid PDF file."}), 400

    text, err = extract_text_from_pdf(file)
    if err:
        return jsonify({"error": err}), 400

    prompt = RECOMMEND_JOB_PROMPT.format(text)
    result, err = query_openrouter(prompt)
    if err:
        return jsonify({"error": err}), 500

    return jsonify({"job_title": result})


@app.route("/analyze", methods=["POST"])
def analyze_resume():
    file = request.files.get("resume")
    job_desc = request.form.get("job_desc", "").strip()

    if not file or not job_desc:
        return jsonify({"error": "Missing resume file or job description."}), 400

    text, err = extract_text_from_pdf(file)
    if err:
        return jsonify({"error": err}), 400

    prompt = ANALYZE_PROMPT.format(text, job_desc)
    result, err = query_openrouter(prompt)
    if err:
        return jsonify({"error": err}), 500

    return jsonify({"suggestions": result})


@app.route("/skill_improve", methods=["POST"])
def skill_improvement():
    file = request.files.get("resume")
    job_desc = request.form.get("job_desc", "").strip()

    if not file or not job_desc:
        return jsonify({"error": "Missing resume or job description."}), 400

    text, err = extract_text_from_pdf(file)
    if err:
        return jsonify({"error": err}), 400

    prompt = SKILL_IMPROVE_PROMPT.format(text, job_desc)
    result, err = query_openrouter(prompt)
    if err:
        return jsonify({"error": err}), 500

    return jsonify({"suggestions": result})


@app.route("/percentage_match", methods=["POST"])
def match_percentage():
    file = request.files.get("resume")
    job_desc = request.form.get("job_desc", "").strip()

    if not file or not job_desc:
        return jsonify({"error": "Missing resume or job description."}), 400

    text, err = extract_text_from_pdf(file)
    if err:
        return jsonify({"error": err}), 400

    prompt = PERCENTAGE_MATCH_PROMPT.format(text, job_desc)
    result, err = query_openrouter(prompt)
    if err:
        return jsonify({"error": err}), 500

    return jsonify({"match_percentage": result})


@app.route("/test", methods=["GET"])
def test():
    return jsonify({"status": "Server is running."})

# --- Run Server ---
if __name__ == "__main__":
    app.run(debug=True, port=5000)


 * Serving Flask app '__main__'
 * Debug mode: on


2025-06-19 17:11:27,152 - INFO - WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.24:5000
2025-06-19 17:11:27,152 - INFO - Press CTRL+C to quit
2025-06-19 17:12:48,270 - WARNING - Not found: /
2025-06-19 17:12:48,272 - INFO - 127.0.0.1 - - [19/Jun/2025 17:12:48] "GET / HTTP/1.1" 404 -
2025-06-19 17:12:56,672 - INFO - 127.0.0.1 - - [19/Jun/2025 17:12:56] "GET /analyze HTTP/1.1" 405 -
2025-06-19 17:14:24,168 - INFO - Extracted 2326 characters from PDF.
2025-06-19 17:14:27,687 - INFO - 127.0.0.1 - - [19/Jun/2025 17:14:27] "POST /analyze HTTP/1.1" 200 -
2025-06-19 17:14:38,589 - WARNING - Not found: /قثؤخةةثىي_تخلا
2025-06-19 17:14:38,593 - INFO - 127.0.0.1 - - [19/Jun/2025 17:14:38] "POST /قثؤخةةثىي_تخلا HTTP/1.1" 404 -
2025-06-19 17:14:55,338 - INFO - Extracted 2326 characters from PDF.
2025-06-19 17:14:55,830 - INF